In [2]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..','..','..'))
from configs import spark_config as config
from utils import spark_utils as utils

spark = utils.create_spark_session('filtering_data')

directory = config.MERGED_DATA_DIR

team12Stats = spark.read.csv(os.path.join(directory, 'team_stats.csv'), header=True, inferSchema=True)
balltoball = spark.read.csv(os.path.join(directory, 'ball_by_ball.csv'), header=True, inferSchema=True)
playerStats = spark.read.csv(os.path.join(directory, 'player_stats.csv'), header=True, inferSchema=True)

/home/ravikumar/miniconda3/envs/t20i/lib/python3.12/site-packages/airflow/configuration.py:859 FutureWarning: section/key [core/sql_alchemy_conn] has been deprecated, you should use[database/sql_alchemy_conn] instead. Please update your `conf.get*` call to use the new name

[2024-11-24T14:57:43.858+0530] {spark_utils.py:17} INFO - Creating Spark session.


your 131072x1 screen size is bogus. expect trouble
24/11/24 14:57:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[2024-11-24T14:57:48.205+0530] {spark_utils.py:37} INFO - Spark session created successfully.


In [3]:
team12Stats.select('match_id').distinct().count(), balltoball.select('match_id').distinct().count(), playerStats.select('match_id').distinct().count()

(3028, 3687, 1183)

In [4]:
# Extract match_id columns from each dataset
team12_match_ids = team12Stats.select('match_id').distinct()
balltoball_match_ids = balltoball.select('match_id').distinct()
player_match_ids = playerStats.select('match_id').distinct()

# Find intersection of match_id across all three datasets
common_match_ids = team12_match_ids.intersect(balltoball_match_ids).intersect(player_match_ids)
common_match_ids.collect()

# convert to list
common_match_ids_list = [row['match_id'] for row in common_match_ids.collect()]

print(common_match_ids_list),len(common_match_ids_list)

[211048, 287874, 352674, 571150, 875549, 966765, 1200428, 1267306, 1383107, 518954, 636165, 1172507, 1200425, 1233464, 1436482, 1436486, 356014, 412690, 569237, 636166, 1199544, 1284495, 1310938, 426417, 1183526, 1199548, 1257948, 1424830, 1439822, 249227, 350347, 995467, 1072320, 1217743, 1310183, 1188623, 1257406, 1273271, 1310940, 1439825, 452154, 1273149, 1327506, 1336049, 1383093, 875553, 1229820, 1257405, 1279382, 1373568, 1400056, 291343, 645647, 660149, 1317137, 1348649, 1383103, 1391339, 1415751, 426418, 439505, 1187677, 287858, 533295, 661697, 1153696, 1197528, 1381458, 473918, 1197526, 1298167, 1418182, 1431086, 533290, 1327508, 1381452, 1415724, 525816, 531635, 1217744, 1263151, 1323551, 1339614, 298804, 546414, 936153, 1188379, 1361774, 885969, 931398, 963697, 225271, 412697, 936155, 951373, 1074968, 1263158, 1298139, 1317487, 1327505, 1403289, 356000, 538068, 1298147, 1336048, 1439819, 287863, 1199542, 1310178, 287866, 426393, 474466, 566926, 895819, 1282748, 1450765, 129

(None, 1031)

In [5]:
# filter team12Stats

filtered_team12Stats = team12Stats.filter(team12Stats.match_id.isin(common_match_ids_list)).drop("_c0")

# filter balltoball

filtered_balltoball = balltoball.filter(balltoball.match_id.isin(common_match_ids_list)).drop("_c0")

# filter playersStats

filtered_playerStats = playerStats.filter(playerStats.match_id.isin(common_match_ids_list)).drop("_c0")

In [6]:
print(filtered_playerStats.count(), filtered_playerStats.select('match_id').distinct().count(), len(filtered_playerStats.columns))
print(filtered_team12Stats.count(), filtered_team12Stats.select('match_id').distinct().count(), len(filtered_team12Stats.columns))
print(filtered_balltoball.count(), filtered_balltoball.select('match_id').distinct().count(), len(filtered_balltoball.columns))

22682 1031 13
1031 1031 14


239273 1031 12


In [7]:
filtered_team12Stats.show(2)
filtered_balltoball.show(2)
filtered_playerStats.show(2)

+--------+------+--------------------+---------------------+---------------------+--------------------+-----------------------+-----------------------+--------------------+---------------------+---------------------+--------------------+-----------------------+-----------------------+
|match_id|gender|Cumulative Won team1|Cumulative Lost team1|Cumulative Tied team1|Cumulative W/L team1|Cumulative AveRPW team1|Cumulative AveRPO team1|Cumulative Won team2|Cumulative Lost team2|Cumulative Tied team2|Cumulative W/L team2|Cumulative AveRPW team2|Cumulative AveRPO team2|
+--------+------+--------------------+---------------------+---------------------+--------------------+-----------------------+-----------------------+--------------------+---------------------+---------------------+--------------------+-----------------------+-----------------------+
|  211028|     0|                   0|                    0|                    0|                 0.0|                    0.0|               

In [8]:
filtered_balltoball = filtered_balltoball.select('match_id','innings','ball',"runs","wickets",'curr_score','curr_wickets','overs','run_rate','required_run_rate','target',"won")

utils.spark_save_data(filtered_team12Stats, config.FILTERED_DATA_DIR, 'team12_stats.csv')
utils.spark_save_data(filtered_balltoball, config.FILTERED_DATA_DIR, 'ball_to_ball.csv')
utils.spark_save_data(filtered_playerStats, config.FILTERED_DATA_DIR, 'players_stats.csv')
spark.stop()

[2024-11-24T14:58:09.816+0530] {spark_utils.py:64} INFO - Successfully wrote data to /usr/ravi/t20/data/4_filteredData/team12_stats.csv


[2024-11-24T14:58:11.339+0530] {spark_utils.py:64} INFO - Successfully wrote data to /usr/ravi/t20/data/4_filteredData/ball_to_ball.csv


[2024-11-24T14:58:12.056+0530] {spark_utils.py:64} INFO - Successfully wrote data to /usr/ravi/t20/data/4_filteredData/players_stats.csv
